In [1]:
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
import pandas as pd
import pprint as pprint 
import os
import pymongo

In [2]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [3]:
# Define database and collection
db = client.apartments_db
collection = db.apartment

In [4]:
def getListOfFiles(dirName):
    # create a list of file and sub directories 
    # names in the given directory 
    listOfFile = os.listdir(dirName)
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = os.path.join(dirName, entry)
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
                
    return allFiles

In [5]:
dirName = 'Resources/Tables'
listoffiles= getListOfFiles(dirName)
listoffiles

['Resources/Tables\\23173_table.csv',
 'Resources/Tables\\23219_table.csv',
 'Resources/Tables\\23220_table.csv',
 'Resources/Tables\\23221_table.csv',
 'Resources/Tables\\23222_table.csv',
 'Resources/Tables\\23223_table.csv',
 'Resources/Tables\\23224_table.csv',
 'Resources/Tables\\23225_table.csv',
 'Resources/Tables\\23226_table.csv',
 'Resources/Tables\\23227_table.csv',
 'Resources/Tables\\23228_table.csv',
 'Resources/Tables\\23229_table.csv',
 'Resources/Tables\\23230_table.csv',
 'Resources/Tables\\23231_table.csv',
 'Resources/Tables\\23233_table.csv',
 'Resources/Tables\\23234_table.csv',
 'Resources/Tables\\23235_table.csv',
 'Resources/Tables\\23236_table.csv',
 'Resources/Tables\\23238_table.csv',
 'Resources/Tables\\23294_table.csv']

In [6]:
Richmond_zipcodes = ['23173', '23219', '23220', '23221', '23222', '23223', \
                     '23224', '23225', '23226', '23227', '23228', '23229', '23230', '23231', '23233','23234', '23235', \
                    '23236', '23238', '23294']
table_dfs =[]
for zipcode in Richmond_zipcodes:
    df_name = zipcode
    table_dfs.append(df_name)
    
table_dfs    

['23173',
 '23219',
 '23220',
 '23221',
 '23222',
 '23223',
 '23224',
 '23225',
 '23226',
 '23227',
 '23228',
 '23229',
 '23230',
 '23231',
 '23233',
 '23234',
 '23235',
 '23236',
 '23238',
 '23294']

In [7]:
list_of_dfs = []
for df, file in zip(table_dfs, listoffiles):
    df = pd.read_csv(file)
    list_of_dfs.append(df)
list_of_dfs[2].head()

,Subject,Unnamed: 1,ZCTA5 23220,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,NaN,NaN,Estimate,NaN,NaN,Margin of Error,Percent,Percent Margin of Error,NaN
1,SEX AND AGE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Total population,NaN,"34,409",NaN,NaN,"+/-1,284","34,409",(X),NaN
3,Male,NaN,"16,717",NaN,NaN,+/-817,48.6%,+/-1.5,NaN
4,Female,NaN,"17,692",NaN,NaN,+/-818,51.4%,+/-1.5,NaN


In [8]:
clean_dfs = []
count = 0
for df in list_of_dfs:
    drop_col = df.drop(['Unnamed: 1', 'Unnamed: 3', 'Unnamed: 4'], axis=1)
    if 'Unnamed: 8' in drop_col:
        print(f'found col 8 in df')
        drop_col = drop_col.drop(columns=['Unnamed: 8'])
    df_rename_col = drop_col.rename(columns={"Subject": "Subject", (f"ZCTA5 {Richmond_zipcodes[count]}"): "Estimate", 'Unnamed: 5': 'Margin of Error', \
                                             'Unnamed: 6': 'Percent', 'Unnamed: 7': 'Percent Margin of Error'})
    df_dropna = df_rename_col.dropna(how='any')
    df_dropna.index.name = Richmond_zipcodes[count]
    count = count + 1
    clean_dfs.append(df_dropna)
    
    
    
    

found col 8 in df
found col 8 in df
found col 8 in df
found col 8 in df
found col 8 in df
found col 8 in df
found col 8 in df
found col 8 in df
found col 8 in df
found col 8 in df
found col 8 in df
found col 8 in df
found col 8 in df
found col 8 in df
found col 8 in df
found col 8 in df
found col 8 in df
found col 8 in df
found col 8 in df


In [9]:
df_rename_col.head()

,Subject,Estimate,Margin of Error,Percent,Percent Margin of Error
0,NaN,Estimate,Margin of Error,Percent,Percent Margin of Error
1,SEX AND AGE,NaN,NaN,NaN,NaN
2,Total population,"17,848",+/-790,"17,848",(X)
3,Male,"8,480",+/-531,47.5%,+/-1.9
4,Female,"9,368",+/-511,52.5%,+/-1.9


In [10]:
clean_dfs[0].head()

,Subject,Estimate,Margin of Error,Percent,Percent Margin of Error
23173,,,,,
1,Total population,"2,209",+/-288,"2,209",(X)
2,Male,953,+/-215,43.10%,+/-6.9
3,Female,"1,256",+/-200,56.90%,+/-6.9
4,Sex ratio (males per 100 females),75.9,+/-21.6,(X),(X)
6,Under 5 years,0,+/-12,0.00%,+/-1.5


In [11]:
tables = []
count = 0
for df in clean_dfs:
    table = df.to_html()
    tables.append(table)
    
    count = count + 1

In [12]:
#Beginning Scrapping

In [13]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [14]:
apartments_url = 'https://www.apartments.com/richmond-va/'
browser.visit(apartments_url)

In [15]:
apartments_html = browser.html
apartments_soup = bs(apartments_html, 'html.parser')

In [16]:
print(apartments_soup.prettify())

<!DOCTYPE html>
<html class="js webgl geolocation cssanimations csstransforms csstransforms3d csstransitions svg" data-placeholder-focus="true" id="appContainer" lang="en" xmlns="http://www.w3.org/1999/xhtml">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=no" name="viewport"/>
  <meta content="telephone=no" name="format-detection"/>
  <meta content="email=no" name="format-detection"/>
  <meta content="true" name="HandheldFriendly"/>
  <title>
   Apartments for Rent in Richmond VA | Apartments.com
  </title>
  <link href="/a/8a55f9/favicon.ico" rel="icon"/>
  <link href="https://www.apartments.com/richmond-va/" rel="canonical"/>
  <link href="https://plus.google.com/+apartmentscom/" rel="publisher"/>
  <meta content="See all 3,435 apartments in Richmond, VA currently available for rent. Each Apartments.com listing has verified availability, rental rates, photos, floor plans and more." name="description"/>
  <me

In [17]:
count = 1
pages = []
for page in range(2):
    page_url = f"{apartments_url}{count}"
    count = count + 1
    pages.append(page_url)
    print(page_url)

    

https://www.apartments.com/richmond-va/1
https://www.apartments.com/richmond-va/2


In [18]:
print(pages)

['https://www.apartments.com/richmond-va/1', 'https://www.apartments.com/richmond-va/2']


In [19]:
apartments = apartments_soup.find_all('article', class_='diamond placard')
# apartment_info = []

In [20]:
def listToString(Zipcode):  
    
    # initialize an empty string 
    str1 = ""  
    
    # traverse in the string   
    for ele in Zipcode:  
        str1 = ele   
    
    # return string   
    return str1

In [21]:
for page in pages:
    browser.visit(page)
    apartments_html = browser.html
    page_soup = bs(apartments_html, 'html.parser')
    apartments = page_soup.find_all('article', class_='diamond placard')
    for apartment in apartments:
        name = apartment.find('a').text
        Name = name.replace('\n','')
        link = apartment.find('a')['href']
        location = apartment.find('div', class_='location').text
        Zipcode = [ele for ele in Richmond_zipcodes if(ele in location)]
        Zipcode_string = listToString(Zipcode)
        for table in tables:
            if Zipcode_string in table:
                table_data = table
        cost = apartment.find('span', class_='altRentDisplay').text

        browser.visit(link)
        apartment_link_html = browser.html
        apartment_link_soup = bs(apartment_link_html, 'html.parser')

        phone = apartment_link_soup.find('span', class_='phoneNumber').text
        Phone_Number = phone.replace('\n\n','').replace('\n','')
        Rating = apartment_link_soup.find('span', class_='rating-tooltip')
        if Rating != None:
            Rating = float(str(Rating).replace('<span class="rating-tooltip">', '').replace(' star property</span>', ''))
        
        school_Type = apartment_link_soup.find('p', class_='schoolType').text
        school_name = apartment_link_soup.find('p', class_='schoolName').text
        grade_levels = apartment_link_soup.find('p', class_='grades').text
        numb_of_students = apartment_link_soup.find('p', class_='numberOfStudents').text
        school_phone = apartment_link_soup.find('p', class_='schoolPhone').text

        apartment_amenities = apartment_link_soup.find_all('section', class_='amenitiesSection') 

        for amenities in apartment_amenities:
            pet_policy = amenities.find_all('div', class_='petPolicyDetails')
            for pet in pet_policy:
                pet = pet.find('span').text
                #if pet == 'Dogs and Cats Allowed': 
                    #print('Dogs and Cats Allowed')
                    #Dogs_allowed = 'Dogs Allowed'
                    #Cats_allowed = 'Cats Allowed'
                #elif pet == 'Dogs and Cats Allowed:': 
                    #print('Dogs + Cats Allowed')
                    #Dogs_allowed = 'Dogs Allowed'
                   # Cats_allowed = 'Cats Allowed'
               # elif pet == 'Dogs Allowed' or 'Dogs Allowed:' and pet == 'Cats Allowed' or 'Cats Allowed:': 
                   # print('Dogs/cats allowed')
                    #Dogs_allowed = 'Dogs Allowed'
                    #Cats_allowed = 'Cats Allowed'
                #else pet == 'No Pets Allowed':
                   # Dogs_allowed = 'No Dogs Allowed'
                   # Cats_allowed = 'No Cats Allowed'
                    

        apartment_info = {'Name': Name, 'Apartment_link' : link, 'Address': location, \
                               'Zipcode': Zipcode_string, 'phone_number': Phone_Number, 'price_range': cost,
                               'rating': Rating, 'Type': school_Type, 'School Name': school_name, 'Grade Levels': grade_levels, \
                               'Student Count': numb_of_students, 'Phone': school_phone, 'Table': table_data}
        collection.insert_one(apartment_info)
#print(name)
#print('--------------------')
#print(link)
#print('--------------------')
#print(location)
#print('--------------------')
#print(phone)
#print('--------------------')
#print(cost)
#print('--------------------')
#print(rating)


In [22]:
apartment_info

{'Name': 'Reflections of West Creek',
 'Apartment_link': 'https://www.apartments.com/reflections-of-west-creek-richmond-va/pd8cl9v/',
 'Address': '4400 Breezy Bay Cir, Richmond, VA 23233',
 'Zipcode': '23233',
 'phone_number': '804-708-3902',
 'price_range': '$1,499',
 'rating': 4.9,
 'Type': 'Public Elementary School',
 'School Name': 'Colonial Trail Elementary School',
 'Grade Levels': 'Grades PK-5',
 'Student Count': '621 Students',
 'Phone': '(804) 364-0055',
 'Table': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Subject</th>\n      <th>Estimate</th>\n      <th>Margin of Error</th>\n      <th>Percent</th>\n      <th>Percent Margin of Error</th>\n    </tr>\n    <tr>\n      <th>23233</th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>2</th>\n      <td>Total population</td>\n      <td>30,680</td>\n      <td>+/-819</td>\n    

In [23]:
for i in apartment_info:
    print(i)

Name
Apartment_link
Address
Zipcode
phone_number
price_range
rating
Type
School Name
Grade Levels
Student Count
Phone
Table
_id


In [24]:
apartments = db.apartment.find()

for apartment in apartments:
    print(apartment)


{'_id': ObjectId('5d3bb96c1b78b3f698e20c4f'), 'Name': 'The Locks Tower', 'Apartment_link': 'https://www.apartments.com/the-locks-tower-richmond-va/c58v4yf/', 'Address': '1001 E Byrd St, Richmond, VA 23219', 'Zipcode': '23219', 'phone_number': '804-205-1868', 'price_range': '$1,124 - 5,650', 'rating': None, 'Type': 'Public Elementary, Middle & High School', 'School Name': 'Richmond Hospital Ep.', 'Grade Levels': 'Grades PK-12', 'Student Count': '', 'Phone': '(804) 828-0426', 'Table': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Subject</th>\n      <th>Estimate</th>\n      <th>Margin of Error</th>\n      <th>Percent</th>\n      <th>Percent Margin of Error</th>\n    </tr>\n    <tr>\n      <th>23219</th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>2</th>\n      <td>Total population</td>\n      <td>4,138</td>\n      <td>+/-524</